# 第1章：基础提示结构

- [课程](#lesson)
- [练习](#exercises)
- [示例游乐场](#example-playground)

## 环境准备

运行下方初始化单元以加载 API Key 并建立 `get_completion` 助手函数。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## 课程

Anthropic 提供两类 API：历史版本的 [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) 与当前的 [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)。本教程将只使用 Messages API。

使用 Messages API 调用 Claude 至少需要以下参数：
- `model`：你要调用的模型的 [API 模型名称](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`：在停止前最多生成的 token 数。注意 Claude 可能在到达最大值前就停止。该参数仅指定上限；同时这是“硬性”停止，可能会在词或句子中途截断。

- `messages`：输入消息数组。模型按 `user` 与 `assistant` 轮流的对话回合进行训练。创建新 `Message` 时，通过该参数提供先前的对话轮次，模型将生成下一条 `Message`。
  - 每条输入消息都需要 `role` 与 `content` 字段。你可以只提供一条 `user` 角色消息，也可以包含多组交替的 `user` 与 `assistant`（必须交替）。第一条消息必须是 `user` 角色。

可选参数包括：
- `system`：系统提示（见下文）。
  
- `temperature`：响应的多样性程度。本课程与练习设置为 0。

完整参数列表请参阅 [API 文档](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html)。

### 示例

下面来看一些格式正确的提示，Claude 会如何回应。对每个单元，运行该单元（`Shift+Enter`），Claude 的回复会显示在其下方。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

接下来看看一些未按 Messages API 正确格式编写的提示。对于这些格式错误的提示，Messages API 会返回错误。

首先是一个在 `messages` 数组中缺少 `role` 和 `content` 字段的调用示例。

> ⚠️ **警告：** 由于提示中 `messages` 参数格式不正确，下面的单元会返回错误。这是预期行为。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"Hi Claude, how are you?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

下面这个示例没有在 `user` 与 `assistant` 角色之间交替。

> ⚠️ **警告：** 由于未在 `user` 与 `assistant` 角色间交替，Claude 将返回错误信息。这是预期行为。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

`user` 与 `assistant` 消息必须交替，且必须以 `user` 回合开始。你可以在一个提示中包含多组 `user` 和 `assistant`（模拟多轮对话）。你也可以在末尾添加一个 `assistant` 消息片段让 Claude 从该处续写（后续章节详述）。

#### 系统提示（System Prompts）

你也可以使用系统提示。系统提示用于在“User”回合给出问题或任务之前，为 Claude 提供上下文、规则与指导。

在结构上，系统提示独立于 `user` 与 `assistant` 列表，放在单独的 `system` 参数中（可参见本笔记本 [环境准备](#setup) 中 `get_completion` 的结构）。

在本教程中，凡使用系统提示的地方，我们都在补全函数中提供了 `system` 字段；如果不需要系统提示，将 `SYSTEM_PROMPT` 置为空字符串即可。

#### 系统提示示例

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

为什么要使用系统提示？**精心编写的系统提示可以提升 Claude 的执行效果**，例如增强其遵循规则与指令的能力。更多信息参阅文档：[如何使用系统提示](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)。

接下来进入练习。如果想在不更改上方内容的前提下试验课程中的提示，请滚动到页面底部，前往[示例游乐场](#example-playground)。

---

## 练习
- [练习 1.1 - 数到三](#exercise-11---counting-to-three)
- [练习 1.2 - 系统提示](#exercise-12---system-prompt)

### 练习 1.1 - 数到三
使用正确的 `user` / `assistant` 格式，编辑下方 `PROMPT` 使 Claude **数到三**。输出会显示你的答案是否正确。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_1_1_hint)

### 练习 1.2 - 系统提示

修改 `SYSTEM_PROMPT`，让 Claude 的回答像一个 3 岁小朋友。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示？运行下方单元！

In [ ]:
print(hints.exercise_1_2_hint)

### 恭喜！

如果你已经完成到目前为止的所有练习，可以进入下一章。祝你玩得开心！

---

## 示例游乐场

这里是自由试验区，你可以复用上面的示例提示并做修改，观察它如何影响 Claude 的输出。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"Hi Claude, how are you?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))